In [1]:
!nvidia-smi

Tue Jul 16 15:07:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
%%writefile sort.cu
#include <cub/cub.cuh>


void SORT(int* nums, size_t n, int* output) {
    void* d_temp_storage = nullptr;
    size_t temp_storage_bytes = 0;
    cub::DeviceRadixSort::SortKeys(d_temp_storage, temp_storage_bytes, nums, output, n);
    cudaMalloc(&d_temp_storage, temp_storage_bytes);
    cub::DeviceRadixSort::SortKeys(d_temp_storage, temp_storage_bytes, nums, output, n);
    cudaFree(d_temp_storage);
}

int main() {
    int num_elements = 1024;
    int* keys = new int[num_elements];
    int* sorted_keys = new int[num_elements];
    for (int i = 0; i < num_elements; ++i) {
        keys[i] = num_elements - i;
    }
    int* d_keys;
    int* d_sorted_keys;
    cudaMalloc(&d_keys, num_elements * sizeof(int));
    cudaMalloc(&d_sorted_keys, num_elements * sizeof(int));
    cudaMemcpy(d_keys, keys, num_elements * sizeof(int), cudaMemcpyHostToDevice);
    SORT(d_keys, num_elements, d_sorted_keys);
    cudaMemcpy(sorted_keys, d_sorted_keys, num_elements * sizeof(int), cudaMemcpyDeviceToHost);
    for (int i = 1; i < num_elements; i++) {
        std::cout << sorted_keys[i] << " ";
    }
    std::cout << std::endl;
    delete[] keys;
    delete[] sorted_keys;
    return 0;
}





Overwriting sort.cu


In [5]:
!nvcc -std=c++14 -arch=sm_75 sort.cu -o sort

In [6]:
!time ./sort

2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 27

In [7]:
%%writefile mergesort.cu
#include <cuda_runtime.h>
#include <cmath>
#include <cassert>
#include <random>
#include <iostream>
using namespace std;
__global__ void f(int *a,int *tmp,int n)
{
    int left=blockIdx.x*n;
    int right=left+n-1;
    int mid=(left+right)/2;
    int i=left,j=mid+1,k=left;
    while((i<=mid)&&(j<=right))
    {
        if(a[i]<=a[j])
        {
            tmp[k]=a[i];
            ++i;++k;
        }
        else
        {
            tmp[k]=a[j];
            ++j;++k;
        }
    }
    if(i>mid)
    {
        for(;j<=right;++j,++k)
        tmp[k]=a[j];
    }
    else
    {
        for(;i<=mid;++i,++k)
        tmp[k]=a[i];
    }
    for(k=left;k<=right;++k) a[k]=tmp[k];
}
int main(){
    int n = 1<<20;
    int *a,*tmp,*deva,*ans;
    mt19937 rnd(0x98213);
    a=(int*)malloc(n*sizeof(int));
    ans=(int*)malloc(n*sizeof(int));
    for(int i=0;i<n;i++) a[i]=rnd()%1000000+1;
    cudaMalloc((void**)&deva,n*sizeof(int));
    cudaMalloc((void**)&tmp,n*sizeof(int));
    cudaMemcpy(deva,a,n*sizeof(int), cudaMemcpyHostToDevice);
    for(int i=2;i<=n;i<<=1) f<<<n/i,1>>>(deva,tmp,i);
    cudaMemcpy(ans,deva,n*sizeof(int),cudaMemcpyDeviceToHost);
    for(int i=0;i<n-1;i++) assert(ans[i]<=ans[i+1]);
    cudaFree(deva);
    cudaFree(tmp);
    free(a);
    free(ans);
    return 0;
}

Writing mergesort.cu


In [8]:
!nvcc -std=c++14 -arch=sm_75 mergesort.cu -o mergesort

In [9]:
!time ./mergesort


real	0m0.669s
user	0m0.453s
sys	0m0.207s


In [32]:
%%writefile new_sort.cu
#include<iostream>
#include<stdio.h>
#include<stdlib.h>
#define NThreads 1024
#define NBlocks 1

#define Num NThreads*NBlocks
__device__ void swap(int &a, int &b){
    int t = a;
    a = b;
    b = t;
}
__global__ void bitonic_sort(int* arr){
    extern __shared__ int shared_arr[];
    const unsigned int tid = blockIdx.x * blockDim.x + threadIdx.x;
    //const unsigned int tid = threadIdx.x;
    shared_arr[tid] = arr[tid];
    __syncthreads();

    //for(int i=2; i<=blociDim.x; i<<=1){
    for(unsigned int i=2; i<=Num; i<<=1){
        for(unsigned int j=i>>1; j>0; j>>=1){
            unsigned int tid_comp = tid ^ j;
            if(tid_comp > tid){
                if((tid & i)==0){ //ascending
                    if(shared_arr[tid]>shared_arr[tid_comp]){
                        swap(shared_arr[tid],shared_arr[tid_comp]);
                    }
                }
                else{ //desending
                    if(shared_arr[tid]<shared_arr[tid_comp]){
                        swap(shared_arr[tid],shared_arr[tid_comp]);
                    }
                }
            }
            __syncthreads();
        }
    }
    arr[tid] = shared_arr[tid];
}

int main(int argc, char* argv[]){
    int* arr= (int*) malloc(Num*sizeof(int));
    for(int i=0;i<Num;i++){
        arr[i] = rand() % 1000;
    }
    int* ptr;
    cudaMalloc((void**)&ptr,Num*sizeof(int));
    cudaMemcpy(ptr,arr,Num*sizeof(int),cudaMemcpyHostToDevice);
    for(int i=0;i<Num;i++){
        printf("%d\t",arr[i]);
    }
    printf("\n");
    dim3 blocks(NBlocks,1);
    dim3 threads(NThreads,1);
    bitonic_sort<<<blocks,threads,Num*sizeof(int)>>>(ptr);
    //bitonic_sort<<<1,Num,Num*sizeof(int)>>>(ptr);
    cudaMemcpy(arr,ptr,Num*sizeof(int),cudaMemcpyDeviceToHost);
    for(int i=0;i<Num;i++){
        printf("%d\t",arr[i]);
    }
    printf("\n");

    cudaFree(ptr);
    return 0;
}


Overwriting new_sort.cu


In [33]:
!nvcc -std=c++14 -arch=sm_75 new_sort.cu -o newsort

In [34]:
!time ./newsort

383	886	777	915	793	335	386	492	649	421	362	27	690	59	763	926	540	426	172	736	211	368	567	429	782	530	862	123	67	135	929	802	22	58	69	167	393	456	11	42	229	373	421	919	784	537	198	324	315	370	413	526	91	980	956	873	862	170	996	281	305	925	84	327	336	505	846	729	313	857	124	895	582	545	814	367	434	364	43	750	87	808	276	178	788	584	403	651	754	399	932	60	676	368	739	12	226	586	94	539	795	570	434	378	467	601	97	902	317	492	652	756	301	280	286	441	865	689	444	619	440	729	31	117	97	771	481	675	709	927	567	856	497	353	586	965	306	683	219	624	528	871	732	829	503	19	270	368	708	715	340	149	796	723	618	245	846	451	921	555	379	488	764	228	841	350	193	500	34	764	124	914	987	856	743	491	227	365	859	936	432	551	437	228	275	407	474	121	858	395	29	237	235	793	818	428	143	11	928	529	776	404	443	763	613	538	606	840	904	818	128	688	369	917	917	996	324	743	470	183	490	499	772	725	644	590	505	139	954	786	669	82	542	464	197	507	355	804	348	611	622	828	299	343	746	568	340	422	311	810	605	801	661	730	878	305